# Library


In [75]:
import os, re, math, cv2, tqdm, shutil
import numpy as np
from scipy.interpolate import interp1d
from pathlib import Path
from collections import Counter, defaultdict
import mediapipe as mp

# Data Augmentation

In [76]:
OUT = Path(r"C:\Users\Jerome\anaconda3\CPE313_MONTOJO\MODIFIABLE - PD\KeypointsAugmented")
for p in OUT.rglob("*.npy"):
    p.unlink()  # delete all old augmented npy files
print("Cleared augmented .npy files.")

Cleared augmented .npy files.


In [77]:
rng = np.random.default_rng(2025)
NUM_LM = 21 # number of landmarks per hand
FEATURE_COORDS = 42 * 3 # 2 hands * 21 landmarks * 3 coords (x,y,z)
APPEND_FLAGS = True
FEATURE_DIM = FEATURE_COORDS + (2 if APPEND_FLAGS else 0) # 128 if appending presence flags
CLIP_LIMIT = 5.0 # max abs value for coords

INPUT_DIR = r"C:\Users\Jerome\anaconda3\CPE313_MONTOJO\MODIFIABLE - PD\KEYPOINTS"
OUTPUT_DIR = r"C:\Users\Jerome\anaconda3\CPE313_MONTOJO\MODIFIABLE - PD\KeypointsAugmented"
os.makedirs(OUTPUT_DIR, exist_ok=True)

SEQUENCE_LENGTH = 48
OVERWRITE = True  # True = regenerate; False = keep existing
PAD_MODE = "zeros"
RIGHT_HAND_ONLY = True

In [78]:
def _f32(x):
    return np.asarray(x, dtype=np.float32)

def split_coords_flags(seq):
    """(T,D)-> coords(T,42,3), flags(T,2). Fill zeros if flags absent."""
    seq = _f32(seq)
    coords = seq[:, :FEATURE_COORDS].reshape(len(seq), 42, 3)
    if APPEND_FLAGS and seq.shape[1] >= FEATURE_COORDS + 2:
        flags = seq[:, FEATURE_COORDS:FEATURE_COORDS+2]
    else:
        flags = np.zeros((len(seq), 2), np.float32)
    return coords, flags

def combine_coords_flags(coords, flags):
    flat = coords.reshape(len(coords), FEATURE_COORDS)
    if APPEND_FLAGS:
        return np.concatenate([flat, flags.astype(np.float32)], axis=1)
    return flat

def swap_left_right(coords, flags):
    """Swap hands 0..20 <-> 21..41 and flags [L,R]->[R,L]."""
    left  = coords[:, :NUM_LM, :].copy()
    right = coords[:, NUM_LM:, :].copy()
    coords[:, :NUM_LM, :] = right
    coords[:, NUM_LM:, :] = left
    if flags.shape[1] == 2:
        flags = flags[:, ::-1]
    return coords, flags

# -------- temporal utilities (mirror extraction) --------
def _motion_mag(coords: np.ndarray) -> np.ndarray:
    v = np.diff(coords, axis=0)
    return np.linalg.norm(v, axis=(1, 2))

def trim_idle_edges(coords: np.ndarray, flags: np.ndarray, max_idle_edge=12, eps=5e-4):
    """Same behavior as extractor: trim long low-motion stretches at start/end."""
    T = coords.shape[0]
    if T <= 2:
        return coords, flags
    v = np.diff(coords, axis=0)
    m = np.linalg.norm(v, axis=(1, 2))           # (T-1,)
    m_full = np.r_[m[0], m, m[-1]] if len(m) > 1 else np.zeros(T, dtype=np.float32)
    active_idx = np.where(m_full > eps)[0]
    if len(active_idx) == 0:
        single = coords[-1:][:1]
        return np.repeat(single, repeats=min(T, 1), axis=0), flags[: min(T, 1)]
    start = max(0, active_idx[0] - 1)
    end   = min(T, active_idx[-1] + 1)
    if start > max_idle_edge: start = max_idle_edge
    if (T - end) > max_idle_edge: end = T - max_idle_edge
    end = max(end, start + 1)
    return coords[start:end], flags[start:end]

def _tail_is_frozen(coords: np.ndarray, k=6, eps=5e-4) -> bool:
    k = min(k, coords.shape[0]-1) if coords.shape[0] > 1 else 1
    tail = coords[-(k+1):]
    diffs = np.abs(np.diff(tail, axis=0)).mean()
    return bool(diffs < eps)

def _defrost_tail(coords: np.ndarray, strength=0.25):
    T = coords.shape[0]
    if T < 4:
        return coords
    seg = max(3, int(T * strength))
    a0 = T - seg
    a1 = T - 1
    if a0 >= a1:
        return coords
    ramp = np.linspace(0.0, 1.0, seg, dtype=np.float32)[:, None, None]
    mid0, mid1 = max(0, T//3), min(T-1, 2*T//3)
    mean_v = (coords[mid1] - coords[mid0]) / max(1, (mid1 - mid0))
    coords[a0:T] = coords[a0:T] + 0.02 * ramp * mean_v
    return coords

def _resample_coords_flags(coords: np.ndarray, flags: np.ndarray, target_len: int):
    """Linear resample coords; nearest-like for flags (re-binarize)."""
    T = coords.shape[0]
    if T == target_len:
        return coords.astype(np.float32), flags.astype(np.float32)
    idx = np.linspace(0, T - 1, num=target_len)
    lo = np.floor(idx).astype(int)
    hi = np.clip(lo + 1, 0, T - 1)
    w  = (idx - lo)[:, None, None]
    coords_out = (1 - w) * coords[lo] + w * coords[hi]
    flags_out  = flags[np.round(idx).astype(int)]
    return coords_out.astype(np.float32), flags_out.astype(np.float32)

def temporal_fix(seq: np.ndarray, target_len: int) -> np.ndarray:
    """
    EXACT match to extractor:
      1) split -> 2) trim idle edges -> 3) resample -> 4) defrost tail if frozen -> 5) combine
    """
    coords, flags = split_coords_flags(seq)
    coords, flags = trim_idle_edges(coords, flags, max_idle_edge=12, eps=5e-4)
    coords, flags = _resample_coords_flags(coords, flags, target_len)
    if _tail_is_frozen(coords, k=6, eps=5e-4):
        coords = _defrost_tail(coords, strength=0.25)
    return combine_coords_flags(coords, flags)

# -------- geometric / noise (no temporal cropping here) --------
def _clamp(coords):
    return np.clip(coords, -CLIP_LIMIT, CLIP_LIMIT).astype(np.float32)

def add_noise(seq, sigma_xy=0.008, sigma_z=0.004):
    seq = _f32(seq).copy()
    coords, flags = split_coords_flags(seq)
    noise = rng.normal(0, [sigma_xy, sigma_xy, sigma_z], size=coords.shape).astype(np.float32)
    coords = _clamp(coords + noise)
    return combine_coords_flags(coords, flags)

def scale_translate(seq, scale_range=(0.9,1.1), translate_xy=(-0.08,0.08)):
    seq = _f32(seq).copy()
    coords, flags = split_coords_flags(seq)
    s  = rng.uniform(*scale_range)
    tx = rng.uniform(*translate_xy)
    ty = rng.uniform(*translate_xy)
    coords = coords * np.array([s,s,s], np.float32)
    coords[...,0] += tx
    coords[...,1] += ty
    coords = _clamp(coords)
    return combine_coords_flags(coords, flags)

def rotate(seq, angle_range=(-10,10)):
    seq = _f32(seq).copy()
    coords, flags = split_coords_flags(seq)
    ang = np.deg2rad(rng.uniform(*angle_range))
    c, s = np.cos(ang), np.sin(ang)
    R = np.array([[c,-s],[s,c]], np.float32)
    coords[..., :2] = coords[..., :2] @ R.T
    coords = _clamp(coords)
    return combine_coords_flags(coords, flags)

def flip(seq):
    """Mirror X + swap hands + swap flags."""
    seq = _f32(seq).copy()
    coords, flags = split_coords_flags(seq)
    coords[...,0] *= -1.0
    coords, flags = swap_left_right(coords, flags)
    coords = _clamp(coords)
    return combine_coords_flags(coords, flags)

def landmark_dropout(seq, p=0.03):
    seq = _f32(seq).copy()
    coords, flags = split_coords_flags(seq)
    mask = rng.random(coords.shape[:-1]) < p  # (T,42)
    coords[mask] = 0.0
    return combine_coords_flags(coords, flags)

def hand_dropout(seq, p=0.10):
    seq = _f32(seq).copy()
    coords, flags = split_coords_flags(seq)
    if rng.random() > p:
        return combine_coords_flags(coords, flags)
    drop_left = rng.random() < 0.5
    if drop_left:
        coords[:, :NUM_LM, :] = 0.0
        flags[:,0] = 0.0
    else:
        coords[:, NUM_LM:, :] = 0.0
        flags[:,1] = 0.0
    return combine_coords_flags(coords, flags)

# ---- augmentation heads that PRESERVE temporal_fix result ----
def augment_from_fixed(seq_fixed):
    """
    seq_fixed: already temporal_fixed to SEQUENCE_LENGTH.
    Apply geometric/noise-only augs; length stays the same.
    """
    s = seq_fixed
    s = rotate(s)
    s = scale_translate(s)
    if rng.random() < 0.9: s = add_noise(s, sigma_xy=0.008, sigma_z=0.004)
    if rng.random() < 0.4: s = landmark_dropout(s, p=0.03)
    if rng.random() < 0.2: s = hand_dropout(s, p=0.10)
    # safety: ensure flags are 0/1
    if s.shape[1] >= FEATURE_COORDS + 2:
        s[:, FEATURE_COORDS:FEATURE_COORDS+2] = (s[:, FEATURE_COORDS:FEATURE_COORDS+2] >= 0.5).astype(np.float32)
    return _f32(s)

def augment_from_fixed_with_flip(seq_fixed):
    s = augment_from_fixed(seq_fixed)
    s = flip(s)
    if s.shape[1] >= FEATURE_COORDS + 2:
        s[:, FEATURE_COORDS:FEATURE_COORDS+2] = (s[:, FEATURE_COORDS:FEATURE_COORDS+2] >= 0.5).astype(np.float32)
    return _f32(s)

In [79]:
classes = [d for d in os.listdir(INPUT_DIR) if os.path.isdir(os.path.join(INPUT_DIR, d))]

for action in classes:
    in_dir  = os.path.join(INPUT_DIR, action)
    out_dir = os.path.join(OUTPUT_DIR, action)
    os.makedirs(out_dir, exist_ok=True)

    for fname in os.listdir(in_dir):
        if not fname.endswith(".npy"):
            continue
        src = os.path.join(in_dir, fname)
        base = fname[:-4]

        # output paths
        orig_out  = os.path.join(out_dir, f"{base}.npy")
        aug_paths = [os.path.join(out_dir, f"{base}_aug{i}.npy") for i in range(1,6)]

        if (not OVERWRITE) and os.path.exists(orig_out) and all(os.path.exists(p) for p in aug_paths):
            print(f"Skipped {fname} (already augmented)")
            continue

        arr = _f32(np.load(src, allow_pickle=False))
        # normalize feature width to match APPEND_FLAGS setting
        if arr.shape[1] == FEATURE_COORDS and APPEND_FLAGS:
            zf = np.zeros((len(arr), 2), np.float32)
            arr = np.concatenate([arr, zf], axis=1)
        elif arr.shape[1] == FEATURE_COORDS + 2 and not APPEND_FLAGS:
            arr = arr[:, :FEATURE_COORDS]
        elif arr.shape[1] not in (FEATURE_COORDS, FEATURE_COORDS + 2):
            print(f"[skip] {fname}: unexpected D={arr.shape[1]}")
            continue

        orig_seq = temporal_fix(arr, SEQUENCE_LENGTH)

        seqs = [orig_seq]  # always keep original
        for _ in range(5):
            seqs.append(augment_from_fixed(orig_seq))

        if not RIGHT_HAND_ONLY:
            # LEFT/RIGHT symmetry augmentation only if deployment allows it
            forced_flip = flip(orig_seq)
            if forced_flip.shape[1] >= FEATURE_COORDS + 2:
                forced_flip[:, FEATURE_COORDS:FEATURE_COORDS+2] = (forced_flip[:, FEATURE_COORDS:FEATURE_COORDS+2] >= 0.5).astype(np.float32)
            seqs.append(forced_flip)

        # Balanced randomized augmentations (no re-trim; preserve time structure)
        seqs.append(augment_from_fixed(orig_seq))
        seqs.append(augment_from_fixed(orig_seq))

        if not RIGHT_HAND_ONLY:
            seqs.append(augment_from_fixed_with_flip(orig_seq))
            seqs.append(augment_from_fixed_with_flip(orig_seq))

        # Save
        for i, s in enumerate(seqs):
            save_path = orig_out if i == 0 else os.path.join(out_dir, f"{base}_aug{i}.npy")
            np.save(save_path, s.astype(np.float32))

        print(f"Processed {fname} -> {len(seqs)} sequences saved")

print("Offline augmentation complete!")

Processed 0.npy -> 8 sequences saved
Processed 1.npy -> 8 sequences saved
Processed 10.npy -> 8 sequences saved
Processed 11.npy -> 8 sequences saved
Processed 12.npy -> 8 sequences saved
Processed 13.npy -> 8 sequences saved
Processed 14.npy -> 8 sequences saved
Processed 15.npy -> 8 sequences saved
Processed 16.npy -> 8 sequences saved
Processed 17.npy -> 8 sequences saved
Processed 18.npy -> 8 sequences saved
Processed 19.npy -> 8 sequences saved
Processed 2.npy -> 8 sequences saved
Processed 20.npy -> 8 sequences saved
Processed 3.npy -> 8 sequences saved
Processed 4.npy -> 8 sequences saved
Processed 5.npy -> 8 sequences saved
Processed 6.npy -> 8 sequences saved
Processed 7.npy -> 8 sequences saved
Processed 8.npy -> 8 sequences saved
Processed 9.npy -> 8 sequences saved
Processed 0.npy -> 8 sequences saved
Processed 1.npy -> 8 sequences saved
Processed 10.npy -> 8 sequences saved
Processed 11.npy -> 8 sequences saved
Processed 12.npy -> 8 sequences saved
Processed 13.npy -> 8 se

In [80]:
# Set DATA_PATH to the directory you want to repair (augmented by default)
DATA_PATH = Path(OUTPUT_DIR)

# Build the 'bad' list (class, filename, shape, dtype) for files with wrong shape/flags
bad = []
for cls_dir in DATA_PATH.iterdir():
    if not cls_dir.is_dir():
        continue
    for npy in cls_dir.glob("*.npy"):
        try:
            arr = np.load(npy, allow_pickle=False)
        except Exception as e:
            bad.append((cls_dir.name, npy.name, "unreadable", str(e)))
            continue

        # ensure 2D
        if arr.ndim != 2:
            arr = np.reshape(arr, (arr.shape[0], -1))

        shape_ok = (arr.shape[1] in (FEATURE_COORDS, FEATURE_DIM)) and (arr.shape[0] == SEQUENCE_LENGTH)
        flags_ok = True
        if arr.shape[1] >= FEATURE_DIM:
            fl = arr[:, FEATURE_COORDS:FEATURE_COORDS+2]
            flags_ok = np.isin(fl, [0.0, 1.0]).all()

        if (not shape_ok) or (not flags_ok):
            bad.append((cls_dir.name, npy.name, arr.shape, str(arr.dtype)))

# Repair pass
for cls, name, shp, dt in bad:
    p = DATA_PATH / cls / name
    try:
        repair_file(p)
        print(f"[fixed] {cls}/{name} from shape={shp}, dtype={dt}")
    except Exception as e:
        print(f"[fail ] {cls}/{name}: {e}")

print(f"Repair done. Fixed {sum('fixed' in line for line in map(str, bad))} files; total flagged: {len(bad)}")

Repair done. Fixed 0 files; total flagged: 0


In [81]:
# If there are bad files from previous cells, fix them then rerun cell 4
def fix_length(seq, target=SEQUENCE_LENGTH):
    seq = np.asarray(seq, dtype=np.float32)
    T, D = seq.shape
    if T == target:
        out = seq
    else:
        idx = np.linspace(0, T-1, target)
        lo = np.floor(idx).astype(int)
        hi = np.minimum(lo+1, T-1)
        w  = (idx - lo)[:, None]
        out = (1-w)*seq[lo] + w*seq[hi]
    if out.shape[1] >= FEATURE_DIM:
        out[:, FEATURE_COORDS:FEATURE_COORDS+2] = (out[:, FEATURE_COORDS:FEATURE_COORDS+2] >= 0.5).astype(np.float32)
    return out.astype(np.float32)

def repair_file(path: Path):
    arr = np.load(path, allow_pickle=False)
    if arr.ndim != 2:
        arr = np.reshape(arr, (arr.shape[0], -1)).astype(np.float32)
    if arr.shape[1] == FEATURE_COORDS and APPEND_FLAGS:
        zf = np.zeros((arr.shape[0], 2), np.float32)
        arr = np.concatenate([arr.astype(np.float32), zf], axis=1)
    elif arr.shape[1] > FEATURE_DIM:
        arr = arr[:, :FEATURE_DIM].astype(np.float32)
    elif arr.shape[1] < FEATURE_DIM:
        padw = FEATURE_DIM - arr.shape[1]
        arr = np.concatenate([arr.astype(np.float32), np.zeros((arr.shape[0], padw), np.float32)], axis=1)
    arr = fix_length(arr, SEQUENCE_LENGTH)
    np.save(path, arr.astype(np.float32))

# run repair on the bad list from step 1
for cls, name, shp, dt in bad:
    p = Path(DATA_PATH, cls, name)
    try:
        repair_file(p)
        print(f"[fixed] {cls}/{name} from shape={shp}, dtype={dt}")
    except Exception as e:
        print(f"[fail ] {cls}/{name}: {e}")

In [82]:
# Diagnostic: summarize augmented outputs
import os, numpy as np
# Use OUTPUT_DIR if defined in the notebook, otherwise fall back to the known path
try:
    OUTPUT_DIR
except NameError:
    OUTPUT_DIR = r"C:\Users\Jerome\anaconda3\CPE313_MONTOJO\MODIFIABLE - PD\KeypointsAugmented"

classes = [d for d in os.listdir(OUTPUT_DIR) if os.path.isdir(os.path.join(OUTPUT_DIR, d))]
per_class = {}
shapes = set()
flag_counts = { 'left': 0, 'right': 0, 'frames': 0 }
total = 0
for cls in classes:
    folder = os.path.join(OUTPUT_DIR, cls)
    cnt = 0
    for f in os.listdir(folder):
        if f.endswith('.npy'):
            cnt += 1
            arr = np.load(os.path.join(folder, f))
            shapes.add(tuple(arr.shape))
            if arr.size:
                flags = np.array(arr)[:, -2:]
                flag_counts['left'] += flags[:,0].sum()
                flag_counts['right'] += flags[:,1].sum()
                flag_counts['frames'] += flags.shape[0]
    per_class[cls] = cnt
    total += cnt

print('per-class augmented counts:', per_class)
print('total augmented .npy files:', total)
print('unique shapes found:', shapes)
if flag_counts['frames']:
    print('left_present fraction:', flag_counts['left'] / flag_counts['frames'])
    print('right_present fraction:', flag_counts['right'] / flag_counts['frames'])
else:
    print('no frames found to compute flag stats')


per-class augmented counts: {"Don't Understand": 168, 'Good Afternoon': 168, 'Good Evening': 176, 'Good Morning': 160, 'Hello': 160, '_webcam_captures': 0}
total augmented .npy files: 832
unique shapes found: {(48, 128)}
left_present fraction: 0.011318108974358974
right_present fraction: 0.5452974759615384


In [83]:
from pathlib import Path
DATA_PATH = r"C:\Users\Jerome\anaconda3\CPE313_MONTOJO\MODIFIABLE - PD\KeypointsAugmented"
classes = ["Good Morning","Good Afternoon","Good Evening","Hello","Don't Understand"]

pairs = { (0,0):0, (1,0):0, (0,1):0, (1,1):0 }
same_cols = True
total = 0

for cls in classes:
    for p in Path(DATA_PATH, cls).glob("*.npy"):
        if "_aug" in p.stem: 
            continue
        arr = np.load(p, allow_pickle=False).astype(np.float32)
        flags = arr[:, -2:].astype(np.int32)
        total += len(flags)
        # count pairs
        for a,b in flags:
            pairs[(a,b)] += 1
        # check if columns are identical
        if same_cols and not (flags[:,0] == flags[:,1]).all():
            same_cols = False

print("pair counts:", pairs)
print("columns identical? ->", same_cols)


pair counts: {(0, 0): 2215, (1, 0): 51, (0, 1): 2720, (1, 1): 6}
columns identical? -> False


In [ ]:
from pathlib import Path
DATA_PATH = r"C:\Users\Jerome\anaconda3\CPE313_MONTOJO\MODIFIABLE - PD\KeypointsAugmented"
left = right = active = 0
files = frames = 0
for p in Path(DATA_PATH).rglob("*.npy"):
    arr = np.load(p, allow_pickle=False).astype(np.float32)
    flags = arr[:, -2:]
    active_mask = (flags.sum(axis=1) > 0)
    active += active_mask.sum()
    left  += flags[active_mask, 0].sum()
    right += flags[active_mask, 1].sum()
    files += 1; frames += len(flags)

print("active frames:", active)
print("left_present (active only):", left/active if active else 0.0)
print("right_present (active only):", right/active if active else 0.0)


active frames: 22181
left_present (active only): 0.02037780082052207
right_present (active only): 0.9817862134259051


In [85]:
import numpy as np, random, glob, os

aug_paths = glob.glob(os.path.join(OUTPUT_DIR, "**", "*.npy"), recursive=True)
assert aug_paths, "No augmented .npy found."

def monotonic_time_ops_ok(seq_before, seq_after):
    # crude: correlation of frame index vs DTW alignment should be positive on average
    # here: just check that first/last frames don't swap (cheap proxy)
    return np.linalg.norm(seq_after[0]-seq_before[0]) < np.linalg.norm(seq_after[-1]-seq_before[0])

bad = 0
for p in random.sample(aug_paths, min(50, len(aug_paths))):
    a = np.load(p)
    assert a.ndim == 2 and a.shape[0] >= 16, f"{p}: shape off {a.shape}"
    C = a[:, :126].reshape(a.shape[0], 42, 3)
    # cheap “temporal plausibility”: middle shouldn’t equal last
    if np.allclose(C[len(C)//2], C[-1]):
        bad += 1
print(f"[OK] temporal_plausibility_bad= {bad}")



[OK] temporal_plausibility_bad= 0


In [86]:
import glob, numpy as np, os, random
paths = glob.glob(os.path.join(OUTPUT_DIR, "**", "*.npy"), recursive=True)
assert paths, "No augmented files written."

# shape + class balance
from collections import Counter
cls_counts = Counter()
for p in random.sample(paths, min(100, len(paths))):
    a = np.load(p)
    assert a.shape == (SEQUENCE_LENGTH, FEATURE_DIM), f"{p}: {a.shape}"
for p in paths:
    cls = os.path.basename(os.path.dirname(p))
    cls_counts[cls] += 1
print("per-class augmented counts:", dict(cls_counts))
print("total augmented .npy files:", len(paths))


per-class augmented counts: {"Don't Understand": 168, 'Good Afternoon': 168, 'Good Evening': 176, 'Good Morning': 160, 'Hello': 160}
total augmented .npy files: 832


In [87]:
try:
    FEATURE_COORDS
except NameError:
    FEATURE_COORDS = 126
try:
    FEATURE_DIM
except NameError:
    FEATURE_DIM = 128
try:
    SEQUENCE_LENGTH
except NameError:
    SEQUENCE_LENGTH = 24

# Augmented directories
AUG_INPUT_DIR = Path(r"C:\Users\Jerome\anaconda3\CPE313_MONTOJO\MODIFIABLE - PD\KeypointsAugmented")  # where aug .npy live
RAW_INPUT_DIR = Path(r"C:\Users\Jerome\anaconda3\CPE313_MONTOJO\MODIFIABLE - PD\KEYPOINTS")           # original extraction dir (for cross-check)
AUG_VIZ_DIR   = Path(r"C:\Users\Jerome\anaconda3\CPE313_MONTOJO\MODIFIABLE - PD\VIZ_AUG")
AUG_VIZ_DIR.mkdir(parents=True, exist_ok=True)

EXPECTED_AUG_SUFFIXES = {"", "_aug1", "_aug2", "_aug3", "_aug4", "_aug5"}  # current code makes 1 original + 5 augs = 6 total per stem

def split_coords_flags(arr: np.ndarray):
    """Return (coords[T,42,3], flags[T,2]) with zero-flags if absent."""
    T, D = arr.shape
    coords = arr[:, :FEATURE_COORDS].reshape(T, 42, 3)
    flags  = arr[:, FEATURE_COORDS:FEATURE_COORDS+2] if D >= FEATURE_COORDS+2 else np.zeros((T,2), np.float32)
    return coords.astype(np.float32), flags.astype(np.float32)

def active_frames_count(flags: np.ndarray):
    """How many frames w/ any hand present."""
    return int((flags.sum(axis=1) > 0).sum())

def left_right_presence(flags: np.ndarray):
    """Fraction of frames where left/right flag > 0.5."""
    T = len(flags)
    if T == 0: 
        return 0.0, 0.0
    left_frac  = float((flags[:,0] > 0.5).mean())
    right_frac = float((flags[:,1] > 0.5).mean())
    return left_frac, right_frac

stem_re = re.compile(r"^(?P<stem>.+?)(?P<suf>(_aug\d+)?)\.npy$")

In [88]:
RIGHT_HAND_ONLY = True  # match your augmentation policy

if RIGHT_HAND_ONLY:
    EXPECTED_AUG_SUFFIXES = {"", "_aug1", "_aug2"}
    EXPECTED_PER_STEM = 3
else:
    EXPECTED_AUG_SUFFIXES = {"", "_aug1", "_aug2", "_aug3", "_aug4", "_aug5"}
    EXPECTED_PER_STEM = 6

AUG_INPUT_DIR = Path(OUTPUT_DIR)  # audit the augmented directory

stem_re = re.compile(r'^(?P<stem>.+?)(?P<suf>_aug\d+)?\.npy$')

def split_coords_flags_audit(seq):
    coords = seq[:, :FEATURE_COORDS].reshape(len(seq), 42, 3)
    flags  = seq[:, FEATURE_COORDS:FEATURE_COORDS+2] if seq.shape[1] >= FEATURE_DIM else np.zeros((len(seq),2), np.float32)
    return coords, flags

def active_frames_count(flags):
    # frames where either hand is present
    return int((flags.sum(axis=1) > 0.5).sum())

def left_right_presence(flags):
    left_frac  = float((flags[:,0] > 0.5).mean()) if len(flags) else 0.0
    right_frac = float((flags[:,1] > 0.5).mean()) if len(flags) else 0.0
    return left_frac, right_frac

# override the split used below to avoid relying on augmentation cell's definition
def split_coords_flags(arr):
    return split_coords_flags_audit(arr)

# == Audit augmented files ==
per_class_counts = Counter()
shape_counts     = Counter()
nan_files        = []
empty_files      = []
bad_shape        = []
total_active     = 0
left_fracs, right_fracs = [], []

# Per class → stem → found suffixes, to detect duplicates/strays (e.g., legacy _aug0)
by_class_stem = defaultdict(lambda: defaultdict(list))

for cls_dir in sorted([p for p in AUG_INPUT_DIR.iterdir() if p.is_dir()]):
    cls = cls_dir.name
    for p in sorted(cls_dir.glob("*.npy")):
        m = stem_re.match(p.name)
        if not m:
            continue
        stem, suf = m.group("stem"), m.group("suf") or ""
        by_class_stem[cls][stem].append(suf)

        try:
            arr = np.load(str(p), allow_pickle=False).astype(np.float32)
        except Exception as e:
            bad_shape.append((str(p), f"load_error:{e}"))
            continue

        if arr.size == 0:
            empty_files.append(str(p))
            continue

        shape_counts[arr.shape] += 1
        if np.isnan(arr).any():
            nan_files.append(str(p))

        if arr.shape[1] not in (FEATURE_COORDS, FEATURE_DIM):
            bad_shape.append((str(p), f"D={arr.shape[1]}"))
            continue
        if arr.shape[0] != SEQUENCE_LENGTH or arr.shape[1] != FEATURE_DIM:
            bad_shape.append((str(p), f"unexpected shape {arr.shape}"))

        coords, flags = split_coords_flags(arr)
        total_active += active_frames_count(flags)
        lf, rf = left_right_presence(flags)
        left_fracs.append(lf)
        right_fracs.append(rf)

        per_class_counts[cls] += 1

# Summaries
total_files = sum(per_class_counts.values())
avg_left    = float(np.mean(left_fracs)) if left_fracs else 0.0
avg_right   = float(np.mean(right_fracs)) if right_fracs else 0.0

print("=== Augmentation Audit ===")
print("Per-class augmented counts:", dict(per_class_counts))
print("Total augmented .npy files:", total_files)
print("Unique shapes found:", dict(shape_counts))
print(f"Left_present fraction (avg over sequences):  {avg_left:.6f}")
print(f"Right_present fraction (avg over sequences): {avg_right:.6f}")
print("Active frames (sum):", total_active)
print("Empty files:", len(empty_files), "| NaN files:", len(nan_files), "| Bad shape/Load issues:", len(bad_shape))

# Detect unexpected suffixes / wrong count per stem
unexpected = []
wrong_cardinality = []
for cls, stems in by_class_stem.items():
    for stem, sufs in stems.items():
        sset = set(sufs)
        # Anything not in EXPECTED_AUG_SUFFIXES?
        extra = sorted(list(sset - EXPECTED_AUG_SUFFIXES))
        if extra:
            unexpected.append((cls, stem, extra))
        # Expected exactly 6 variants per source stem
        if len(sset) != EXPECTED_PER_STEM:
            wrong_cardinality.append((cls, stem, sorted(sufs)))

print("\n-- Anomalies --")
print("Unexpected suffixes (e.g., legacy _aug0):", len(unexpected))
if unexpected:
    for cls, stem, extra in unexpected[:10]:
        print(f"  [{cls}] {stem}: extras={extra}  (showing first 10)")

print("Stems with wrong #files (≠6):", len(wrong_cardinality))
if wrong_cardinality:
    for cls, stem, sufs in wrong_cardinality[:10]:
        print(f"  [{cls}] {stem}: found {len(set(sufs))} variants -> {sufs}  (first 10)")

=== Augmentation Audit ===
Per-class augmented counts: {"Don't Understand": 168, 'Good Afternoon': 168, 'Good Evening': 176, 'Good Morning': 160, 'Hello': 160}
Total augmented .npy files: 832
Unique shapes found: {(48, 128): 832}
Left_present fraction (avg over sequences):  0.011318
Right_present fraction (avg over sequences): 0.545297
Active frames (sum): 22181
Empty files: 0 | NaN files: 0 | Bad shape/Load issues: 0

-- Anomalies --
Unexpected suffixes (e.g., legacy _aug0): 104
  [Don't Understand] 0: extras=['_aug3', '_aug4', '_aug5', '_aug6', '_aug7']  (showing first 10)
  [Don't Understand] 1: extras=['_aug3', '_aug4', '_aug5', '_aug6', '_aug7']  (showing first 10)
  [Don't Understand] 10: extras=['_aug3', '_aug4', '_aug5', '_aug6', '_aug7']  (showing first 10)
  [Don't Understand] 11: extras=['_aug3', '_aug4', '_aug5', '_aug6', '_aug7']  (showing first 10)
  [Don't Understand] 12: extras=['_aug3', '_aug4', '_aug5', '_aug6', '_aug7']  (showing first 10)
  [Don't Understand] 13: ex

In [89]:
# == Optional cleanup: move unexpected files out ==
TRASH = AUG_INPUT_DIR.parent / "KeypointsAugmented_trash"
TRASH.mkdir(exist_ok=True)

moved = 0
for cls, stems in by_class_stem.items():
    cls_dir = AUG_INPUT_DIR / cls
    trash_cls = TRASH / cls
    trash_cls.mkdir(parents=True, exist_ok=True)
    for stem, sufs in stems.items():
        sset = set(sufs)
        # Move unexpected suffixes
        extras = sset - EXPECTED_AUG_SUFFIXES
        for suf in extras:
            src = cls_dir / f"{stem}{suf}.npy"
            if src.exists():
                dst = trash_cls / src.name
                src.rename(dst)
                moved += 1
        # Also enforce exactly one of each expected; if duplicates, move the duplicates
        for suf in EXPECTED_AUG_SUFFIXES:
            matches = sorted(cls_dir.glob(f"{stem}{suf}.npy"))
            if len(matches) > 1:
                for dup in matches[1:]:
                    dst = trash_cls / dup.name
                    dup.rename(dst)
                    moved += 1

print(f"Moved {moved} unexpected/duplicate files to: {TRASH}")

Moved 520 unexpected/duplicate files to: C:\Users\Jerome\anaconda3\CPE313_MONTOJO\MODIFIABLE - PD\KeypointsAugmented_trash


In [90]:
# ---- knobs ----
AUG_INPUT_DIR = Path(r"C:\Users\Jerome\anaconda3\CPE313_MONTOJO\MODIFIABLE - PD\KeypointsAugmented")
AUG_VIZ_DIR   = Path(r"C:\Users\Jerome\anaconda3\CPE313_MONTOJO\MODIFIABLE - PD\VIZ_AUG")
AUG_VIZ_DIR.mkdir(parents=True, exist_ok=True)

# Set to True if you want to re-generate videos even when they already exist.
OVERWRITE_VIZ = True

# Optional: render only specific suffixes; set to None to render everything found.
# Examples: ONLY_SUFFIXES = {"", "_aug1", "_aug2", "_aug3", "_aug4", "_aug5"}
ONLY_SUFFIXES = {"_aug1","_aug2","_aug3","_aug4","_aug5"}

# Optional: filter by class names; set to None to include all.
ONLY_CLASSES = None  # e.g., {"Hello", "Good Morning"}

# ---- renderer config (same look as before) ----
CANVAS_W, CANVAS_H = 1080, 1080
MARGIN, POINT_R, THICK = 40, 4, 2
FONT = cv2.FONT_HERSHEY_SIMPLEX
INPUT_DURATION_SEC = 8.0
SAVE_MP4, PREVIEW = True, False

HAND_CONNECTIONS = [
    (0,1),(1,2),(2,3),(3,4),
    (0,5),(5,6),(6,7),(7,8),
    (5,9),(9,10),(10,11),(11,12),
    (9,13),(13,14),(14,15),(15,16),
    (13,17),(17,18),(18,19),(19,20),
    (0,17)
]

FEATURE_COORDS = 126  # 42*3
def split_coords_flags(arr: np.ndarray):
    T, D = arr.shape
    coords = arr[:, :FEATURE_COORDS].reshape(T, 42, 3).astype(np.float32)
    flags  = (arr[:, FEATURE_COORDS:FEATURE_COORDS+2] if D >= FEATURE_COORDS+2 else np.zeros((T,2), np.float32)).astype(np.float32)
    return coords, flags

def _minmax_xy(coords):
    xy = coords[..., :2].reshape(-1, 2)
    mask = ~(np.isclose(xy[:,0], 0.0) & np.isclose(xy[:,1], 0.0))
    xy = xy[mask]
    if xy.size == 0:
        return -1.0, 1.0, -1.0, 1.0
    return xy[:,0].min(), xy[:,0].max(), xy[:,1].min(), xy[:,1].max()

def _project(ptx, pty, x0, y0, scale):
    px = int(x0 + ptx * scale)
    py = int(y0 + pty * scale)
    return px, py

def draw_hand(img, pts21, color, x0, y0, scale):
    if pts21 is None: return
    if np.allclose(pts21, 0.0): return
    for a, b in HAND_CONNECTIONS:
        ax, ay = _project(pts21[a,0], pts21[a,1], x0, y0, scale)
        bx, by = _project(pts21[b,0], pts21[b,1], x0, y0, scale)
        cv2.line(img, (ax, ay), (bx, by), color, THICK, cv2.LINE_AA)
    for i in range(21):
        px, py = _project(pts21[i,0], pts21[i,1], x0, y0, scale)
        cv2.circle(img, (px, py), POINT_R, color, -1, cv2.LINE_AA)

def make_frame(img, t, T, left_on, right_on):
    txt = f"Frame {t+1}/{T} | Left:{'Y' if left_on else 'N'}  Right:{'Y' if right_on else 'N'}"
    cv2.putText(img, txt, (16, 30), FONT, 0.8, (255,255,255), 2, cv2.LINE_AA)
    bar_h = 20
    bar_y0 = CANVAS_H - bar_h - 10
    bar_y1 = CANVAS_H - 10
    cv2.rectangle(img, (MARGIN, bar_y0), (CANVAS_W - MARGIN, bar_y1), (50,50,50), -1)
    x0 = MARGIN
    x1 = CANVAS_W - MARGIN
    xpos = int(x0 + (x1 - x0) * (t / max(1, T-1)))
    cv2.line(img, (xpos, bar_y0), (xpos, bar_y1), (255,255,255), 2, cv2.LINE_AA)

def render_npy_to_mp4(npy_path: Path, out_mp4_path: Path):
    arr = np.load(str(npy_path)).astype(np.float32)
    coords, flags = split_coords_flags(arr)  # (T,42,3) and (T,2)
    T = coords.shape[0]
    L = coords[:, :21, :]
    R = coords[:, 21:, :]

    xmin, xmax, ymin, ymax = _minmax_xy(coords)
    dx = max(1e-6, xmax - xmin)
    dy = max(1e-6, ymax - ymin)
    scale_x = (CANVAS_W - 2*MARGIN) / dx
    scale_y = (CANVAS_H - 2*MARGIN) / dy
    scale = 0.9 * min(scale_x, scale_y)
    cx, cy = CANVAS_W // 2, CANVAS_H // 2

    play_fps = T / max(1e-6, INPUT_DURATION_SEC)

    writer = None
    if SAVE_MP4:
        out_mp4_path.parent.mkdir(parents=True, exist_ok=True)
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        writer = cv2.VideoWriter(str(out_mp4_path), fourcc, play_fps, (CANVAS_W, CANVAS_H))

    if PREVIEW:
        cv2.namedWindow("NPY Visualization (AUG)", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("NPY Visualization (AUG)", 900, 900)

    for t in range(T):
        img = np.zeros((CANVAS_H, CANVAS_W, 3), dtype=np.uint8)
        left_on  = flags[t,0] > 0.5
        right_on = flags[t,1] > 0.5
        draw_hand(img, L[t], (255,255,0),  cx, cy, scale)  # Left = cyan
        draw_hand(img, R[t], (0,165,255), cx, cy, scale)   # Right = orange
        make_frame(img, t, T, left_on, right_on)
        cv2.putText(img, "Left = cyan, Right = orange (normalized coords)", (16, CANVAS_H-40),
                    FONT, 0.6, (200,200,200), 1, cv2.LINE_AA)
        if writer is not None: writer.write(img)
        if PREVIEW:
            key = cv2.waitKey(int(1000 / max(1, int(round(play_fps))))) & 0xFF
            if key == 27: break

    if writer is not None: writer.release()
    if PREVIEW: cv2.destroyAllWindows()

# ---- enumerate and render everything ----
import re
stem_re = re.compile(r"^(?P<stem>.+?)(?P<suf>(_aug\d+)?)\.npy$")

total = 0
skipped = 0
rendered = 0

for cls_dir in sorted([p for p in AUG_INPUT_DIR.iterdir() if p.is_dir()]):
    cls = cls_dir.name
    if ONLY_CLASSES and cls not in ONLY_CLASSES:
        continue
    out_dir = AUG_VIZ_DIR / cls
    out_dir.mkdir(parents=True, exist_ok=True)

    for npy in sorted(cls_dir.glob("*.npy")):
        m = stem_re.match(npy.name)
        if not m:
            continue
        suf = m.group("suf") or ""
        if ONLY_SUFFIXES is not None and suf not in ONLY_SUFFIXES:
            continue

        out_mp4 = out_dir / f"{npy.stem}_viz.mp4"
        total += 1
        if out_mp4.exists() and not OVERWRITE_VIZ:
            skipped += 1
            continue
        try:
            render_npy_to_mp4(npy, out_mp4)
            rendered += 1
        except Exception as e:
            print(f"[error] {cls}/{npy.name}: {e}")

print("\n=== Render summary ===")
print(f"Found:    {total}")
print(f"Rendered: {rendered}")
print(f"Skipped (already existed): {skipped}")
print(f"Saved to: {AUG_VIZ_DIR}")


=== Render summary ===
Found:    208
Rendered: 208
Skipped (already existed): 0
Saved to: C:\Users\Jerome\anaconda3\CPE313_MONTOJO\MODIFIABLE - PD\VIZ_AUG


In [91]:
# CLEAN THE MIRRORS AUGMENTATION
AUG_INPUT_DIR = Path(r"C:\Users\Jerome\anaconda3\CPE313_MONTOJO\MODIFIABLE - PD\KeypointsAugmented")
TRASH = AUG_INPUT_DIR.parent / "KeypointsAugmented_trash"
TRASH.mkdir(exist_ok=True)

moved = 0
for cls_dir in sorted([p for p in AUG_INPUT_DIR.iterdir() if p.is_dir()]):
    trash_cls = TRASH / cls_dir.name
    trash_cls.mkdir(parents=True, exist_ok=True)
    for p in cls_dir.glob("*_mirror.npy"):
        shutil.move(str(p), str(trash_cls / p.name))
        moved += 1

print(f"Moved {moved} *_mirror.npy files to: {TRASH}")

Moved 0 *_mirror.npy files to: C:\Users\Jerome\anaconda3\CPE313_MONTOJO\MODIFIABLE - PD\KeypointsAugmented_trash


In [92]:
RAW_DIR = Path(r"C:\Users\Jerome\anaconda3\CPE313_MONTOJO\MODIFIABLE - PD\KEYPOINTS")
AUG_DIR = Path(r"C:\Users\Jerome\anaconda3\CPE313_MONTOJO\MODIFIABLE - PD\KeypointsAugmented")
OUT_DIR = Path(r"C:\Users\Jerome\anaconda3\CPE313_MONTOJO\MODIFIABLE - PD\VIZ_COMPARE")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# ---- filters / knobs ----
ONLY_CLASSES   = None  # e.g., {"Hello"} to restrict; None = all classes
ONLY_SUFFIXES  = {"", "_aug1","_aug2","_aug3","_aug4","_aug5"}  # which augmented variants to compare
OVERWRITE_VIZ  = False
INPUT_DURATION_SEC = 4.0  # playback length per clip (slower -> smoother for inspection)

# ---- drawing config ----
CANVAS_W, CANVAS_H = 720, 720
PAD_BETWEEN = 20
MARGIN, POINT_R, THICK = 40, 3, 2
FONT = cv2.FONT_HERSHEY_SIMPLEX
SAVE_MP4, PREVIEW = True, False

# ---- data constants/helpers ----
FEATURE_COORDS = 126  # 42*3
SEQUENCE_LENGTH = 24  # expected model input length

def split_coords_flags(arr: np.ndarray):
    T, D = arr.shape
    coords = arr[:, :FEATURE_COORDS].reshape(T, 42, 3).astype(np.float32)
    flags  = (arr[:, FEATURE_COORDS:FEATURE_COORDS+2] if D >= FEATURE_COORDS+2 else np.zeros((T,2), np.float32)).astype(np.float32)
    return coords, flags

def resize_sequence(seq: np.ndarray, length: int):
    """Linear resample to 'length'; binarize flags if present."""
    seq = np.asarray(seq, dtype=np.float32)
    T = len(seq)
    if T == 0:
        W = seq.shape[1] if seq.ndim == 2 else (FEATURE_COORDS+2)
        return np.zeros((length, W), np.float32)
    if T == length:
        out = seq
    else:
        idx = np.linspace(0, T-1, length)
        lo = np.floor(idx).astype(int)
        hi = np.minimum(lo+1, T-1)
        w  = (idx - lo)[:,None]
        out = (1-w)*seq[lo] + w*seq[hi]
    if out.shape[1] >= FEATURE_COORDS + 2:
        out[:, FEATURE_COORDS:FEATURE_COORDS+2] = (out[:, FEATURE_COORDS:FEATURE_COORDS+2] >= 0.5).astype(np.float32)
    return out.astype(np.float32)

HAND_CONNECTIONS = [
    (0,1),(1,2),(2,3),(3,4),
    (0,5),(5,6),(6,7),(7,8),
    (5,9),(9,10),(10,11),(11,12),
    (9,13),(13,14),(14,15),(15,16),
    (13,17),(17,18),(18,19),(19,20),
    (0,17)
]

def _minmax_xy(coords):
    xy = coords[..., :2].reshape(-1, 2)
    mask = ~(np.isclose(xy[:,0], 0.0) & np.isclose(xy[:,1], 0.0))
    xy = xy[mask]
    if xy.size == 0:
        return -1.0, 1.0, -1.0, 1.0
    return xy[:,0].min(), xy[:,0].max(), xy[:,1].min(), xy[:,1].max()

def _project(ptx, pty, x0, y0, scale):
    px = int(x0 + ptx * scale)
    py = int(y0 + pty * scale)
    return px, py

def draw_hand(img, pts21, color, x0, y0, scale):
    if pts21 is None: return
    if np.allclose(pts21, 0.0): return
    for a, b in HAND_CONNECTIONS:
        ax, ay = _project(pts21[a,0], pts21[a,1], x0, y0, scale)
        bx, by = _project(pts21[b,0], pts21[b,1], x0, y0, scale)
        cv2.line(img, (ax, ay), (bx, by), color, THICK, cv2.LINE_AA)
    for i in range(21):
        px, py = _project(pts21[i,0], pts21[i,1], x0, y0, scale)
        cv2.circle(img, (px, py), POINT_R, color, -1, cv2.LINE_AA)

def make_panel(arr: np.ndarray, title: str):
    """Returns a function that draws frame t of arr onto a fresh panel image."""
    coords, flags = split_coords_flags(arr)
    T = coords.shape[0]
    L = coords[:, :21, :]
    R = coords[:, 21:, :]

    xmin, xmax, ymin, ymax = _minmax_xy(coords)
    dx = max(1e-6, xmax - xmin)
    dy = max(1e-6, ymax - ymin)
    scale_x = (CANVAS_W - 2*MARGIN) / dx
    scale_y = (CANVAS_H - 2*MARGIN) / dy
    scale = 0.9 * min(scale_x, scale_y)
    cx, cy = CANVAS_W // 2, CANVAS_H // 2

    def draw(t: int):
        img = np.zeros((CANVAS_H, CANVAS_W, 3), dtype=np.uint8)
        left_on  = flags[t,0] > 0.5
        right_on = flags[t,1] > 0.5
        draw_hand(img, L[t], (255,255,0),  cx, cy, scale)  # Left = cyan
        draw_hand(img, R[t], (0,165,255), cx, cy, scale)   # Right = orange
        cv2.putText(img, title, (16, 34), FONT, 0.9, (255,255,255), 2, cv2.LINE_AA)
        cv2.putText(img, f"Frame {t+1}/{T} | L:{'Y' if left_on else 'N'} R:{'Y' if right_on else 'N'}",
                    (16, CANVAS_H-16), FONT, 0.6, (200,200,200), 1, cv2.LINE_AA)
        return img
    return draw, T

stem_re = re.compile(r"^(?P<stem>.+?)(?P<suf>(_aug\d+)?)\.npy$")

def render_side_by_side(raw_path: Path, aug_path: Path, out_path: Path, label_left="RAW", label_right="AUG"):
    raw = np.load(str(raw_path)).astype(np.float32)
    aug = np.load(str(aug_path)).astype(np.float32)

    # For fair visual comparison, resample BOTH to SEQUENCE_LENGTH
    raw_r = resize_sequence(raw, SEQUENCE_LENGTH)
    aug_r = resize_sequence(aug, SEQUENCE_LENGTH)

    left_draw, TL  = make_panel(raw_r, label_left)
    right_draw, TR = make_panel(aug_r, label_right)
    T = max(TL, TR)  # should both be SEQUENCE_LENGTH

    play_fps = T / max(1e-6, INPUT_DURATION_SEC)
    H = CANVAS_H
    W = CANVAS_W*2 + PAD_BETWEEN

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out_path.parent.mkdir(parents=True, exist_ok=True)
    writer = cv2.VideoWriter(str(out_path), fourcc, play_fps, (W, H))

    for t in range(T):
        left  = left_draw(min(t, TL-1))
        right = right_draw(min(t, TR-1))
        canvas = np.zeros((H, W, 3), dtype=np.uint8)
        canvas[:, :CANVAS_W] = left
        canvas[:, CANVAS_W+PAD_BETWEEN:] = right
        # vertical divider
        cv2.rectangle(canvas, (CANVAS_W, 0), (CANVAS_W+PAD_BETWEEN, H), (25,25,25), -1)
        writer.write(canvas)
        if PREVIEW:
            cv2.imshow("Compare RAW vs AUG", canvas)
            if cv2.waitKey(int(1000/max(1,int(round(play_fps))))) & 0xFF == 27:
                break
    writer.release()
    if PREVIEW:
        cv2.destroyAllWindows()

# ---- Batch over classes/stems and compare every augmentation ----
total_pairs = 0
skipped = 0
rendered = 0

classes = sorted([p.name for p in RAW_DIR.iterdir() if p.is_dir()])
if ONLY_CLASSES:
    classes = [c for c in classes if c in ONLY_CLASSES]

for cls in classes:
    raw_cls = RAW_DIR / cls
    aug_cls = AUG_DIR / cls
    if not aug_cls.exists():
        print(f"[warn] missing augmented class folder: {aug_cls}")
        continue
    out_cls = OUT_DIR / cls
    out_cls.mkdir(parents=True, exist_ok=True)

    # Map stems present in RAW
    raw_files = sorted(raw_cls.glob("*.npy"))
    for raw_p in raw_files:
        stem = raw_p.stem  # e.g., "12"
        # list all aug for this stem
        aug_list = []
        for aug_p in sorted(aug_cls.glob(f"{stem}*.npy")):
            m = stem_re.match(aug_p.name)
            if not m: 
                continue
            suf = m.group("suf") or ""
            if ONLY_SUFFIXES is not None and suf not in ONLY_SUFFIXES:
                continue
            aug_list.append((aug_p, suf))

        if not aug_list:
            continue

        for aug_p, suf in aug_list:
            label_right = "AUG" + (suf if suf else " (orig)")
            out_name = f"{stem}{suf}_COMPARE.mp4" if suf else f"{stem}_COMPARE.mp4"
            out_path = out_cls / out_name
            total_pairs += 1
            if out_path.exists() and not OVERWRITE_VIZ:
                skipped += 1
                continue
            try:
                render_side_by_side(raw_p, aug_p, out_path, label_left="RAW", label_right=label_right)
                rendered += 1
            except Exception as e:
                print(f"[error] {cls}/{raw_p.name} vs {aug_p.name}: {e}")

print("\n=== Compare summary ===")
print(f"Pairs found: {total_pairs}")
print(f"Rendered:    {rendered}")
print(f"Skipped:     {skipped}")
print(f"Saved to:    {OUT_DIR}")


=== Compare summary ===
Pairs found: 474
Rendered:    0
Skipped:     474
Saved to:    C:\Users\Jerome\anaconda3\CPE313_MONTOJO\MODIFIABLE - PD\VIZ_COMPARE


In [93]:
AUG_DIR         = Path(r"C:\Users\Jerome\anaconda3\CPE313_MONTOJO\MODIFIABLE - PD\KeypointsAugmented")
SEQUENCE_LENGTH = 48
APPEND_FLAGS    = True
FEATURE_COORDS  = 42 * 3
FEATURE_DIM     = FEATURE_COORDS + (2 if APPEND_FLAGS else 0)  # 128
CLIP_LIMIT      = 5.0
EPS_MOTION      = 5e-4

# ---- helpers (mirror extraction logic) ----
def split_coords_flags(arr: np.ndarray):
    T, D = arr.shape
    coords = arr[:, :FEATURE_COORDS].reshape(T, 42, 3)
    if APPEND_FLAGS and D >= FEATURE_COORDS + 2:
        flags = arr[:, FEATURE_COORDS:FEATURE_COORDS+2]
    else:
        flags = np.zeros((T,2), np.float32)
    return coords, flags

def tail_is_frozen(coords: np.ndarray, k=6, eps=EPS_MOTION) -> bool:
    k = min(k, coords.shape[0]-1) if coords.shape[0] > 1 else 1
    tail = coords[-(k+1):]
    if tail.shape[0] < 2: 
        return True
    diffs = np.abs(np.diff(tail, axis=0)).mean()
    return bool(diffs < eps)

def motion_spread_pass(coords: np.ndarray, eps=EPS_MOTION, min_ratio=0.25) -> bool:
    """Pass if at least min_ratio of frame-to-frame motion exceeds eps."""
    if coords.shape[0] < 2: 
        return False
    v = np.linalg.norm(np.diff(coords, axis=0), axis=(1,2))
    return (v > eps).mean() >= min_ratio

# ---- scan & validate ----
per_class_counts = Counter()
issues = {
    "wrong_shape": [],
    "coord_out_of_range": [],
    "flag_non_binary": [],
    "flag_nan": [],
}
pair_counts = Counter()   # (L_flag, R_flag)
det_ratio_accum = []
frozen_tail_flags = []
motion_spread_flags = []

classes = [d for d in sorted(os.listdir(AUG_DIR)) if (AUG_DIR / d).is_dir()]
total_files = 0
unique_shapes = set()

for cls in classes:
    cdir = AUG_DIR / cls
    files = sorted([p for p in cdir.glob("*.npy")])
    per_class_counts[cls] += len(files)
    for f in files:
        total_files += 1
        arr = np.load(f, allow_pickle=False)
        unique_shapes.add(arr.shape)

        # 1) shape check
        if arr.shape != (SEQUENCE_LENGTH, FEATURE_DIM):
            issues["wrong_shape"].append((str(f), arr.shape))
            # skip deeper checks for wrong-shaped files
            continue

        coords, flags = split_coords_flags(arr)

        # 2) coord range check
        max_abs = np.abs(coords).max(initial=0.0)
        if not np.isfinite(max_abs) or (max_abs > (CLIP_LIMIT + 1e-5)):
            issues["coord_out_of_range"].append((str(f), float(max_abs)))

        # 3) flags sanity: binary & non-NaN
        if np.isnan(flags).any():
            issues["flag_nan"].append(str(f))
        # allow small numeric noise, but force round for check
        fb = np.round(flags).astype(np.int32)
        if not np.array_equal(flags, fb.astype(flags.dtype)):
            # tolerate tiny floating error by thresholding at 0.5
            fx = (flags >= 0.5).astype(np.float32)
            if not np.array_equal(fx, flags):
                issues["flag_non_binary"].append(str(f))
            flags = fx  # keep binarized version for downstream statistics

        # 4) detection ratio & pair stats
        det = (flags.sum(axis=1) > 0).astype(np.int32)
        det_ratio_accum.append(det.mean())

        for l, r in flags:
            pair_counts[(int(l), int(r))] += 1

        # 5) tail frozen & motion spread quick checks
        frozen_tail_flags.append(tail_is_frozen(coords, k=6, eps=EPS_MOTION))
        motion_spread_flags.append(motion_spread_pass(coords, eps=EPS_MOTION, min_ratio=0.25))

# ---- report ----
print("\n=== Augmentation Verification Summary ===")
print(f"Aug dir: {AUG_DIR}")
print(f"Classes: {classes}")
print(f"Total .npy files: {total_files}")
print(f"Unique shapes found: {unique_shapes}")

print("\nPer-class file counts:")
for k,v in per_class_counts.items():
    print(f"  {k:<18} {v}")

# pair stats
total_pairs = sum(pair_counts.values()) or 1
lp = (pair_counts[(1,0)] + pair_counts[(1,1)]) / total_pairs
rp = (pair_counts[(0,1)] + pair_counts[(1,1)]) / total_pairs
print("\nFlag presence fractions (across all frames):")
print(f"  left_present : {lp:.4f}")
print(f"  right_present: {rp:.4f}")
print(f"  pair counts  : {dict(pair_counts)}")

# detection & motion stats
if det_ratio_accum:
    det_avg = float(np.mean(det_ratio_accum))
    print(f"\nAvg detection ratio per sequence (any hand present): {det_avg:.3f}")
else:
    print("\nAvg detection ratio per sequence: n/a")

frozen_rate = np.mean(frozen_tail_flags) if frozen_tail_flags else float('nan')
spread_rate = np.mean(motion_spread_flags) if motion_spread_flags else float('nan')
print(f"Frozen tail fraction: {frozen_rate:.3f} (should be ~0.0)")
print(f"Motion spread pass  : {int(np.sum(motion_spread_flags))}/{len(motion_spread_flags)}")

# issues
any_issues = any(len(v) for v in issues.values())
print("\nIssues:")
for k, lst in issues.items():
    print(f"  {k}: {len(lst)}")
if any_issues:
    print("\nExample problems (up to 5 each):")
    for k, lst in issues.items():
        if not lst: 
            continue
        print(f"\n{k}:")
        for item in lst[:5]:
            print(" -", item)
else:
    print("  None")

print("\nDone.")



=== Augmentation Verification Summary ===
Aug dir: C:\Users\Jerome\anaconda3\CPE313_MONTOJO\MODIFIABLE - PD\KeypointsAugmented
Classes: ["Don't Understand", 'Good Afternoon', 'Good Evening', 'Good Morning', 'Hello', '_webcam_captures']
Total .npy files: 312
Unique shapes found: {(48, 128)}

Per-class file counts:
  Don't Understand   63
  Good Afternoon     63
  Good Evening       66
  Good Morning       60
  Hello              60
  _webcam_captures   0

Flag presence fractions (across all frames):
  left_present : 0.0114
  right_present: 0.5461
  pair counts  : {(0, 0): 6646, (1, 0): 152, (0, 1): 8160, (1, 1): 18}

Avg detection ratio per sequence (any hand present): 0.556
Frozen tail fraction: 0.000 (should be ~0.0)
Motion spread pass  : 312/312

Issues:
  wrong_shape: 0
  coord_out_of_range: 3
  flag_non_binary: 0
  flag_nan: 0

Example problems (up to 5 each):

coord_out_of_range:
 - ("C:\\Users\\Jerome\\anaconda3\\CPE313_MONTOJO\\MODIFIABLE - PD\\KeypointsAugmented\\Don't Underst

In [94]:
# TO FIX IF THERE IS COORD OUT OF RANGE, AFTER RUNNING THIS RE-RUN PREVIOUS CELL
AUG_DIR = Path(r"C:\Users\Jerome\anaconda3\CPE313_MONTOJO\MODIFIABLE - PD\KeypointsAugmented")
FEATURE_COORDS = 42*3
CLIP_LIMIT = 5.0

def split_coords_flags(arr):
    T, D = arr.shape
    coords = arr[:, :FEATURE_COORDS].reshape(T, 42, 3)
    flags  = arr[:, FEATURE_COORDS:] if D > FEATURE_COORDS else np.zeros((T,2), np.float32)
    return coords, flags

fixed = 0
for f in AUG_DIR.rglob("*.npy"):
    arr = np.load(f, allow_pickle=False)
    coords, flags = split_coords_flags(arr)
    max_abs = np.abs(coords).max()
    if not np.isfinite(max_abs): 
        continue
    if max_abs > CLIP_LIMIT + 1e-6:
        coords = np.clip(coords, -CLIP_LIMIT, CLIP_LIMIT).astype(np.float32)
        arr = np.concatenate([coords.reshape(len(coords), FEATURE_COORDS), flags.astype(np.float32)], axis=1).astype(np.float32)
        np.save(f, arr)
        fixed += 1

print(f"Clamped & saved {fixed} files.")

Clamped & saved 3 files.
